In [1]:
import pandas as pd
import numpy as np

In [2]:
Index_week=pd.read_table(r"C:\Users\USER\Desktop\論文\原始資料上市下市\大盤資料(週).txt",encoding = 'ANSI')
Index_mon=pd.read_table(r"C:\Users\USER\Desktop\論文\原始資料上市下市\大盤資料(月).txt",encoding = 'ANSI')

In [3]:
print(Index_week.info())
print(Index_mon.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1571 entries, 0 to 1570
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   證券代碼       1571 non-null   object 
 1   簡稱         1571 non-null   object 
 2   年月日        1571 non-null   int64  
 3   收盤價(元)     1571 non-null   float64
 4   成交量(千股)_週  1571 non-null   int64  
 5   報酬率％_週     1571 non-null   float64
 6   週轉率％_週     1571 non-null   float64
 7   報酬率-Ln_週   1571 non-null   float64
dtypes: float64(4), int64(2), object(2)
memory usage: 98.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 364 entries, 0 to 363
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   證券代碼        364 non-null    object 
 1   簡稱          364 non-null    object 
 2   年月          364 non-null    object 
 3   收盤價(元)      364 non-null    float64
 4   成交量(百萬股)_月  364 non-null    int64  
 5   報酬率％_月      364 non-null    fl

In [4]:
Index_week

,證券代碼,簡稱,年月日,收盤價(元),成交量(千股)_週,報酬率％_週,週轉率％_週,報酬率-Ln_週
0,Y9999,加權指數,19930109,3254.05,1622000,-3.6425,1.9225,-3.7105
1,Y9999,加權指數,19930116,3362.52,2048000,3.3334,2.4816,3.2790
2,Y9999,加權指數,19930119,3420.62,893000,1.7279,1.0790,1.7131
3,Y9999,加權指數,19930130,3374.56,1149000,-1.3465,1.3915,-1.3557
4,Y9999,加權指數,19930206,3645.41,4480000,8.0262,5.3500,7.7204
...,...,...,...,...,...,...,...,...
1566,Y9999,加權指數,20230331,15868.06,23563154,-0.2931,1.6653,-0.2935
1567,Y9999,加權指數,20230407,15836.50,9290264,-0.1989,0.6367,-0.1991
1568,Y9999,加權指數,20230414,15929.43,27139884,0.5868,2.0838,0.5851
1569,Y9999,加權指數,20230421,15602.99,29070002,-2.0494,2.2916,-2.0706


In [5]:
Index_mon.rename(columns={'收盤價(元)': '大盤收盤價(元)',
                            "成交量(百萬股)_月":"大盤成交量(百萬股)_月",
                            '報酬率％_月':"大盤報酬率％_月","週轉率％_月":
                            "大盤週轉率％_月","報酬率-Ln_月":"大盤報酬率-Ln_月"},inplace=True)

Index_week.rename(columns={'收盤價(元)': '大盤收盤價(元)',
                            "成交量(千股)_週":"大盤成交量(千股)",
                            '報酬率％_週':"大盤報酬率％",
                           "週轉率％_週":"大盤週轉率％",
                           "報酬率-Ln_週":"大盤報酬率-Ln"},inplace=True)

In [6]:
#將年月取出
Index_week['年月日']=Index_week['年月日'].astype(str)
Index_week['年月']=Index_week['年月日'].str[0:4]+'/'+Index_week['年月日'].str[4:6]
#根據年月分組算出月的標準差
monstd=Index_week.groupby('年月')['大盤報酬率％'].agg(np.std,ddof = 0)
monstd=monstd.shift(1)
Index_mon=pd.merge(Index_mon,monstd,on='年月',how='left')
Index_mon.rename(columns={'大盤報酬率％': "大盤前月標準差"},inplace=True)

#加入年標準差
year=Index_week["年月"].apply(lambda x: x[:4])    
Index_week["年"]=year

Index_mon["年"]=Index_mon["年月"].apply(lambda x: x[:4])    


year_std=Index_week.groupby("年")["大盤報酬率％"].agg(np.std,ddof = 0)
year_std=year_std.shift(1)

Index_mon=pd.merge(Index_mon,year_std,on='年',how='left')
Index_mon.rename(columns={'大盤報酬率％': "大盤前一年標準差"},inplace=True)

In [7]:
Index_mon

,證券代碼,簡稱,年月,大盤收盤價(元),大盤成交量(百萬股)_月,大盤報酬率％_月,大盤週轉率％_月,大盤報酬率-Ln_月,大盤前月標準差,年,大盤前一年標準差
0,Y9999,加權指數,1993/01,3374.56,5712,-0.0740,6.8695,-0.0741,NaN,1993,NaN
1,Y9999,加權指數,1993/02,4384.67,23174,29.9331,26.0032,26.1849,2.700776,1993,NaN
2,Y9999,加權指數,1993/03,4825.29,35473,10.0491,40.2933,9.5756,1.344015,1993,NaN
3,Y9999,加權指數,1993/04,4563.52,22984,-5.4250,27.4669,-5.5777,5.215440,1993,NaN
4,Y9999,加權指數,1993/05,4267.90,16689,-6.4779,20.2142,-6.6972,5.501600,1993,NaN
...,...,...,...,...,...,...,...,...,...,...,...
359,Y9999,加權指數,2022/12,14137.69,118510,-4.9860,8.1836,-5.1144,2.310340,2022,2.496332
360,Y9999,加權指數,2023/01,15265.20,60430,7.9752,3.8716,7.6732,1.135453,2023,2.476506
361,Y9999,加權指數,2023/02,15503.79,92517,1.5631,6.5045,1.5509,0.988860,2023,2.476506
362,Y9999,加權指數,2023/03,15868.06,127541,2.3494,9.1207,2.3225,2.056122,2023,2.476506


In [8]:
Index_week.drop(['年月','年'],axis=1,inplace=True)

In [9]:
Index_mon

,證券代碼,簡稱,年月,大盤收盤價(元),大盤成交量(百萬股)_月,大盤報酬率％_月,大盤週轉率％_月,大盤報酬率-Ln_月,大盤前月標準差,年,大盤前一年標準差
0,Y9999,加權指數,1993/01,3374.56,5712,-0.0740,6.8695,-0.0741,NaN,1993,NaN
1,Y9999,加權指數,1993/02,4384.67,23174,29.9331,26.0032,26.1849,2.700776,1993,NaN
2,Y9999,加權指數,1993/03,4825.29,35473,10.0491,40.2933,9.5756,1.344015,1993,NaN
3,Y9999,加權指數,1993/04,4563.52,22984,-5.4250,27.4669,-5.5777,5.215440,1993,NaN
4,Y9999,加權指數,1993/05,4267.90,16689,-6.4779,20.2142,-6.6972,5.501600,1993,NaN
...,...,...,...,...,...,...,...,...,...,...,...
359,Y9999,加權指數,2022/12,14137.69,118510,-4.9860,8.1836,-5.1144,2.310340,2022,2.496332
360,Y9999,加權指數,2023/01,15265.20,60430,7.9752,3.8716,7.6732,1.135453,2023,2.476506
361,Y9999,加權指數,2023/02,15503.79,92517,1.5631,6.5045,1.5509,0.988860,2023,2.476506
362,Y9999,加權指數,2023/03,15868.06,127541,2.3494,9.1207,2.3225,2.056122,2023,2.476506


In [10]:
Index_mon.isnull().sum()

證券代碼             0
簡稱               0
年月               0
大盤收盤價(元)         0
大盤成交量(百萬股)_月     0
大盤報酬率％_月         0
大盤週轉率％_月         0
大盤報酬率-Ln_月       0
大盤前月標準差          1
年                0
大盤前一年標準差        12
dtype: int64

In [11]:
Index_mon.to_csv(r'C:\Users\USER\Desktop\論文\程式碼\周資料月處理\整理資料\大盤資料(月)_整理.txt',sep='\t', index=False)